# Natural Language to SQL query on QFabric dataset
This notebooks presents a complete pipeline to convert natural language user queries into SQL queries and retrieve corresponding the information from the database and generate a natural language response based on the retrieved information. <br>
<u><b>About Dataset</b></u><br>
QFabric is an open-source change detection dataset with 450,000 change polygons annotated across 504 locations in 100 different cities covering a wide range of geographies and urban fabrics. QFabric is a temporal multi-task dataset with 6 change types and 9 change status classes.
<br>
***The original dataset has been transformed from geojson format to SQLite database. The code for the same is present in <u>geojson_to_sql.ipynb</u>***

<b>We will be using open ai gpt-3.5 turbo model for NL to SQL conversion</b>


## Install dependencies and Imports

In [ ]:
!pip install langchain -U langchain-openai

In [2]:
# This can be omitted if not using gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import sqlite3
from datetime import datetime
import re

## Load llm

In [4]:
# We will be using gpt-3.5 turbo model as llm
llm_gpt = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key= "YOUR OPENAI API KEY")

## Generate SQL queries from Natural Language
In this section we will only convert the natural language user queries into SQL queries by providing appropriate prompt to gpt-3.5 model. Here we will only analyse the quality of SQL queries generated by llm and in later half of the this notebook we will see the complete pipeline implementation to generate the final response.<br><br>
In the <b>zero_shot_vs_few_shot_NL_to_SQL.ipynb</b> notebook we have already seen different prompting strategies to generate SQL queries from NL, and concluded few shot prompting as best performing strategy.

In [5]:
# Few Shot prompt template
template = """### Instructions:
Your task is convert a question into a SQL query, given a SQlite database schema.
Adhere to these rules:
- **Only generate SQL query and no other text
- **Deliberately go through the question and database schema word by word** to appropriately  generate the SQL query for the question
- **You can use Julianday in SQlite database queries
- **Always use aggregation functions like COUNT, SUM etc
- **Appropriately understand the context of each column in the given schema.
### Input:
This query will run on a database whose schema is represented in this string:
CREATE TABLE QFabric (
  id INTEGER PRIMARY KEY, -- Unique ID for each location
  change_type  TEXT, -- Type of the construction. ['commercial', 'demolition', 'industrial', 'mega projects', 'residential', 'road']
  change_status_date1  TEXT, -- construction status at first recorded date. Can be equal to any one from this list ['construction done', 'construction midway', 'construction started', 'excavation', 'greenland', 'land cleared', 'materials dumped', 'NA', 'operational', 'prior construction']
  change_status_date2  TEXT, -- construction status at second recorded date. Can be equal to any one from this list ['construction done', 'construction midway', 'construction started', 'excavation', 'greenland', 'land cleared', 'materials dumped', 'NA', 'operational', 'prior construction']
  change_status_date3  TEXT, -- construction status at third recorded date. Can be equal to any one from this list ['construction done', 'construction midway', 'construction started', 'excavation', 'greenland', 'land cleared', 'materials dumped', 'NA', 'operational', 'prior construction']
  change_status_date4  TEXT, -- construction status at fourth recorded date. Can be equal to any one from this list ['construction done', 'construction midway', 'construction started', 'excavation', 'greenland', 'land cleared', 'materials dumped', 'NA', 'operational', 'prior construction']
  change_status_date5  TEXT, -- construction status at last (fifth) recorded date. Can be equal to any one from this list ['construction done', 'construction midway', 'construction started', 'excavation', 'greenland', 'land cleared', 'materials dumped', 'NA', 'operational', 'prior construction']
  date1  DATE, -- first recorded date
  date2  DATE, -- second recorded date
  date3  DATE, -- third recorded date
  date4  DATE, -- fourth recorded date
  date5  DATE, -- last (fifth) recorded date
  urban_types  TEXT, -- type of urban area like "dense urban", "sparse urban", "rural", "urban slum" etc.
  geography_types  TEXT, -- type of geograpy of the region. For ex. river, forest, green land etc.
  geometry_type  TEXT, -- region geometry, always "polygon"
  coordinates  TEXT -- coordinates of the polygon
);

Below are a number of examples of questions and their corresponding SQL queries. Use the same format as mentioned below

User input: How many houses were constructed after 2015?
SQL query:
    SELECT COUNT(*)
    FROM QFabric
    WHERE 'construction done' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5)
    AND CASE
        WHEN 'construction done' = change_status_date1 THEN date1
        WHEN 'construction done' = change_status_date2 THEN date2
        WHEN 'construction done' = change_status_date3 THEN date3
        WHEN 'construction done' = change_status_date4 THEN date4
        WHEN 'construction done' = change_status_date5 THEN date5
    END > '2015-12-31'
    AND change_type = 'residential'

User input: Is there a house that took at least three years to construct?
SQL query:
  SELECT COUNT(*)
  FROM QFabric
  WHERE
  'construction done' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5)
  AND
  'construction started' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5)
  AND
  (julianday(
            CASE
              WHEN 'construction done'= change_status_date1 THEN date1
              WHEN 'construction done' = change_status_date2 THEN date2
              WHEN 'construction done' = change_status_date3 THEN date3
              WHEN 'construction done' = change_status_date4 THEN date4
              WHEN 'construction done' = change_status_date5 THEN date5
            END
          ) -
  julianday(
            CASE
              WHEN 'construction started'= change_status_date1 THEN date1
              WHEN 'construction started' = change_status_date2 THEN date2
              WHEN 'construction started' = change_status_date3 THEN date3
              WHEN 'construction started' = change_status_date4 THEN date4
              WHEN 'construction started' = change_status_date5 THEN date5
            END
  )) >= 1095
  AND
  change_type = 'residential'

User Input: How many houses are their in rural areas?
SQL Query:
SELECT COUNT(*) FROM QFabric WHERE change_type = 'residential' AND urban_types LIKE '%rural%'

### Response:
User input: {question}
SQL query:
"""

In [6]:
def generate_sql(question, model = llm_gpt, template = template):
  '''
   Generates SQL query from NL questions
  '''

  # prompt template
  prompt = PromptTemplate(template=template, input_variables=["question"])

  # Langchain llm chain
  llm_chain = LLMChain(prompt=prompt, llm=model)

  output = llm_chain.invoke(question)['text']

  return output

def print_generated_sql(query_string):
  '''
  This Helper function prints the llm generated sql query in a better readable format.
  '''
  # Split the query string into lines using '\n'
  lines = query_string.split('\n')

  # Print each line
  for line in lines:
      print(line)

def formated_sql_query(sql_query):
  '''
  This Helper function removes the trailing spaces and unecessay characters like '\n' from the generated sql query
  and makes the query string suitable for Sqlite DB inference.
  '''
  final_sql_query = sql_query.replace('\n', ' ')
  final_sql_query = re.sub(r'\+', ' ', final_sql_query)


  return final_sql_query


In [46]:
# Example query 1
question = "How many commercial buildings became operational after 2016 ?"
sql_query = generate_sql(question = question)
print_generated_sql(sql_query)

SELECT COUNT(*) 
FROM QFabric 
WHERE 'operational' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5) 
AND CASE 
    WHEN 'operational' = change_status_date1 THEN date1 
    WHEN 'operational' = change_status_date2 THEN date2 
    WHEN 'operational' = change_status_date3 THEN date3 
    WHEN 'operational' = change_status_date4 THEN date4 
    WHEN 'operational' = change_status_date5 THEN date5 
END > '2016-12-31' 
AND change_type = 'commercial'


## Final Pipeline
This section includes the implementation of the end to end pipeline which executes below steps:-<br>
1. Convert user natural language question to SQL query
2. Formatize the generated SQL query and extract the corresponding information from the database
3. Generate the final natural language response for the user based on the retrieved information


In [7]:
# Enter the path of the database file
database_path = "/content/drive/MyDrive/Granular Assignment/QFabric_v0.db"


In [10]:
def pipeline(question, database_path):

  # Load the Sqlite database
  con = sqlite3.connect(database_path)
  cur = con.cursor()

  # Generate the SQL query
  gen_sql_query = generate_sql(question)

  # Format the generated sql string into executable string format
  sql_query = formated_sql_query(gen_sql_query)

  # Retrieve information from database
  res = cur.execute(sql_query)
  output_info = res.fetchall()

  # generate final NL response
  prompt_f = """
  ### Instructions:
  Your task is to give a short answer to the given question based on the provided context.
  Below are a number of examples of questions with context and their answers. Use the same format as mentioned below:

  User Input: How many houses were constructed after 2015?
  Context: [(100129,)]
  Answer: There are total 100129 constructed after 2015.

  User Input: Is there a house that took at least seven years to construct?
  Context: [(4018,)]
  Answer: Yes, there are total of 4018 houses that took at least seven years to construct.

  ### Response:
  User Input: {question}
  Context: {context}
  Answer:
  """
  # Final prompt template
  promptf = PromptTemplate(template=prompt_f, input_variables=["question", "context"])

  # Langchain llm chain
  llm_chain_f = LLMChain(prompt=promptf, llm=llm_gpt)
  output_f = llm_chain_f.predict(question = question, context = str(output_info))
  return output_f


## Inference on Sample questions

#### Question 1
<b>How many large scale projects were constructed between 2015 and 2019?</b>

In [11]:
# Print the final answer for the question
question = "How many large scale projects were constructed between 2015 and 2019?"
ans = pipeline(question, database_path = database_path)
print(ans)

There are a total of 61 large scale projects that were constructed between 2015 and 2019.


In [12]:
# Print the corresponding SQL query for the input question
sql_query = generate_sql(question)
print_generated_sql(sql_query)

SELECT COUNT(*)
FROM QFabric 
WHERE 'construction done' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5) 
AND CASE 
    WHEN 'construction done' = change_status_date1 THEN date1 
    WHEN 'construction done' = change_status_date2 THEN date2 
    WHEN 'construction done' = change_status_date3 THEN date3 
    WHEN 'construction done' = change_status_date4 THEN date4 
    WHEN 'construction done' = change_status_date5 THEN date5 
END BETWEEN '2015-01-01' AND '2019-12-31'
AND change_type = 'mega projects'


#### Question 2
<b>How many highways are half constructed till 2015 ?</b>

In [13]:
# Print the final answer for the question
question = "How many highways are half constructed till 2015 ?"
ans = pipeline(question, database_path = database_path)
print(ans)

There are a total of 2086 highways that are half constructed till 2015.


In [14]:
# Print the corresponding SQL query for the input question
sql_query = generate_sql(question)
print_generated_sql(sql_query)

SELECT COUNT(*) 
FROM QFabric 
WHERE ('construction midway' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5) 
OR 'construction started' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5))
AND CASE 
    WHEN 'construction midway' = change_status_date1 THEN date1 
    WHEN 'construction midway' = change_status_date2 THEN date2 
    WHEN 'construction midway' = change_status_date3 THEN date3 
    WHEN 'construction midway' = change_status_date4 THEN date4 
    WHEN 'construction midway' = change_status_date5 THEN date5 
    WHEN 'construction started' = change_status_date1 THEN date1 
    WHEN 'construction started' = change_status_date2 THEN date2 
    WHEN 'construction started' = change_status_date3 THEN date3 
    WHEN 'construction started' = change_status_date4 THEN date4 
    WHEN 'construction started' = change_status_date5 THEN date5 
END <= '2015-12-31' 
AND chan

#### Question 3
<b>How many commercial buildings became operational after 2016 ?</b>

In [15]:
# Print the final answer for the question
question = "How many commercial buildings became operational after 2016 ?"
ans = pipeline(question, database_path = database_path)
print(ans)

There are a total of 479 commercial buildings that became operational after 2016.


In [16]:
# Print the corresponding SQL query for the input question
sql_query = generate_sql(question)
print_generated_sql(sql_query)

SELECT COUNT(*)
FROM QFabric
WHERE 'operational' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5)
AND CASE
    WHEN 'operational' = change_status_date1 THEN date1
    WHEN 'operational' = change_status_date2 THEN date2
    WHEN 'operational' = change_status_date3 THEN date3
    WHEN 'operational' = change_status_date4 THEN date4
    WHEN 'operational' = change_status_date5 THEN date5
END > '2016-12-31'
AND change_type = 'commercial'


#### Question 4
<b>How many highways were constructed withtin 4 years? </b>

In [19]:
# Print the final answer for the question
question = "How many roads were constructed withtin 4 years? "
ans = pipeline(question, database_path = database_path)
print(ans)

There are a total of 476 roads constructed within 4 years.


In [20]:
# Print the corresponding SQL query for the input question
sql_query = generate_sql(question)
print_generated_sql(sql_query)

SELECT COUNT(*) 
FROM QFabric 
WHERE 'construction done' IN (change_status_date1, change_status_date2, change_status_date3, change_status_date4, change_status_date5) 
AND change_type = 'road' 
AND 
(julianday(
            CASE 
              WHEN 'construction done'= change_status_date1 THEN date1
              WHEN 'construction done' = change_status_date2 THEN date2
              WHEN 'construction done' = change_status_date3 THEN date3
              WHEN 'construction done' = change_status_date4 THEN date4
              WHEN 'construction done' = change_status_date5 THEN date5
            END
          ) - 
  julianday(
            CASE 
              WHEN 'construction started'= change_status_date1 THEN date1
              WHEN 'construction started' = change_status_date2 THEN date2
              WHEN 'construction started' = change_status_date3 THEN date3
              WHEN 'construction started' = change_status_date4 THEN date4
              WHEN 'construction started' = change_s

#### Question 5
<b>How many commercial buildings are constructed in densely urban regions?</b>

In [21]:
# Print the final answer for the question
question = "How many commercial buildings are constructed in densely urban regions?"
ans = pipeline(question, database_path = database_path)
print(ans)

There are a total of 28715 commercial buildings constructed in densely urban regions.


In [22]:
# Print the corresponding SQL query for the input question
sql_query = generate_sql(question)
print_generated_sql(sql_query)

SELECT COUNT(*)
FROM QFabric
WHERE change_type = 'commercial' AND urban_types LIKE '%dense urban%'
